In [1]:
!pip install --upgrade azureml-sdk

In [9]:
%%writefile score.py
import json
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import os
import io

# Define class labels
CLASS_NAMES = ["ant", "bee"]

# Load model at initialization
def init():
    global model
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR", ""), "vgg16_hymenoptera.pth")
    
    print(f"AZUREML_MODEL_DIR: {os.getenv('AZUREML_MODEL_DIR')}")
    print(f"Model path: {model_path}")
    
    if not os.path.exists(model_path):
        print("Error: Model file not found!")
        return
    
    model = models.vgg16(pretrained=False)
    model.classifier[6] = nn.Linear(4096, 2)  # Adjust for 2 classes
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    model.eval()
    print("Model loaded successfully.")

# Inference function
def run(raw_data):
    try:
        print("Received data for inference.")
        
        # Parse JSON input
        data = json.loads(raw_data)
        image_bytes = data.get("image")  # Expect base64 or binary image data
        
        if image_bytes is None:
            return json.dumps({"error": "No image found in request."})
        
        print("Image data received.")
        
        # Convert image bytes to PIL Image
        image = Image.open(io.BytesIO(bytearray(image_bytes))).convert("RGB")

        # Preprocess image
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        img_tensor = transform(image).unsqueeze(0)  # Add batch dimension

        # Perform inference
        with torch.no_grad():
            outputs = model(img_tensor)
            _, predicted = torch.max(outputs, 1)
        
        # Return human-readable class label
        predicted_label = CLASS_NAMES[predicted.item()]
        print(f"Predicted label: {predicted_label}")
        return json.dumps({"prediction": predicted_label})

    except Exception as e:
        print(f"Error during inference: {e}")
        return json.dumps({"error": str(e)})

# Run test when executed as a script
if __name__ == "__main__":
    print("Initializing model...")
    init()
    
    # Test inference with a sample image
    test_image_path = "test_image.jpg"  # Replace with an actual image file for testing
    if os.path.exists(test_image_path):
        with open(test_image_path, "rb") as f:
            image_bytes = f.read()
        
        print("Running inference test...")
        response = run(json.dumps({"image": list(image_bytes)}))
        print("Inference result:", response)
    else:
        print(f"Test image '{test_image_path}' not found. Please add a test image to verify inference.")


Overwriting score.py


In [10]:
!python score.py

Initializing model...
AZUREML_MODEL_DIR: None
Model path: vgg16_hymenoptera.pth
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/janak-tutorial/code/Users/janak/Ant_Bees/score.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pyt

In [16]:
%%writefile environment.yml
name: pytorch-env
dependencies:
  - python=3.8
  - joblib
  - pip
  - pip:
      - torch==2.1.0
      - torchvision==0.16.0
      - pillow
      - azureml-sdk
      - azure-ai-ml  # Required for SDK v2
      - azureml-defaults  # ✅ Required for deployment
      - azureml-inference-server-http  # ✅ Required for scoring
      - fastapi  # ✅ Required for API
      - uvicorn  # ✅ Required for API server


Overwriting environment.yml


In [ ]:
# name: mnist-env
# dependencies:
#   - python=3.10
#   - scikit-learn
#   - joblib
#   - pip
#   - pip:
#       - azure-ai-ml  # Required for SDK v2
#       - azureml-defaults  # ✅ Required for deployment
#       - azureml-inference-server-http  # ✅ Required for scoring
#       - fastapi  # ✅ Required for API
#       - uvicorn  # ✅ Required for API server

In [19]:
%%writefile deploy.py
from azureml.core import Workspace, Model, Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Load the Azure ML workspace
ws = Workspace.from_config()

# Get registered model
model = Model(ws, name="vgg16_hymenoptera")

# Define environment
env = Environment.from_conda_specification(name="pytorch-env", file_path="environment.yml")

# Define inference config
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# Define deployment config for Azure Container Instance (ACI)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

# Deploy model as a web service
service = Model.deploy(
    workspace=ws,
    name="vgg16-serviceee",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)

# Print deployment info
print(f"Service State: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")


Overwriting deploy.py


In [20]:
!python deploy.py

/mnt/batch/tasks/shared/LS_root/mounts/clusters/janak-tutorial/code/Users/janak/Ant_Bees/deploy.py:21: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-03-23 12:55:53+00:00 Creating Container Registry if not exists.
2025-03-23 12:55:56+00:00 Building image..
2025-03-23 13:08:55+00:00 Ge